
hightemp.txtは，日本の最高気温の記録を「都道府県」「地点」「℃」「日」のタブ区切り形式で格納したファイルである．以下の処理を行うプログラムを作成し，hightemp.txtを入力ファイルとして実行せよ．さらに，同様の処理をUNIXコマンドでも実行し，プログラムの実行結果を確認せよ．


In [6]:
import subprocess
from pprint import pprint


hightemp = 'data/02-unix/hightemp.txt'

def system(command, **options):
    proc = subprocess.Popen(command, shell=True,
                            stdout=subprocess.PIPE,
                            **options)
    return proc.communicate()[0].decode('utf-8')

10. 行数のカウント
行数をカウントせよ．確認にはwcコマンドを用いよ．


In [7]:
with open(hightemp) as f:
    linecount = 0
    for line in f:
        linecount += 1

assert linecount == int(system(f'wc -l {hightemp}').split()[0]), 'not match'
    

11. タブをスペースに置換
タブ1文字につきスペース1文字に置換せよ．確認にはsedコマンド，trコマンド，もしくはexpandコマンドを用いよ．


In [8]:
with open(hightemp) as f:
    text = f.read()

tab_to_space = text.replace('\t', ' ')
assert tab_to_space == system(f"sed s/$'\t'/$' '/g {hightemp}"), 'not match(1)'
assert tab_to_space == system(f"tr '\t' ' ' < {hightemp}"), 'not match(2)'
assert tab_to_space == system(f"expand -t 1 {hightemp}"), 'not match(3)'



12. 1列目をcol1.txtに，2列目をcol2.txtに保存
各行の1列目だけを抜き出したものをcol1.txtに，2列目だけを抜き出したものをcol2.txtとしてファイルに保存せよ．確認にはcutコマンドを用いよ．

In [9]:
with open(hightemp) as f:
    text = f.read()[:-1] # 最後の改行が邪魔なため
lines = text.split('\n')
col1 = [line.split('\t')[0] for line in lines]
col2 = [line.split('\t')[1] for line in lines]
col1_f = 'data/02-unix/col1.txt'
col2_f = 'data/02-unix/col2.txt'
with open(col1_f, 'w') as f:
    f.write('\n'.join(col1))
with open(col2_f, 'w') as f:
    f.write('\n'.join(col2))

with open(col1_f, 'r') as f:
    assert f.read() + '\n' == system(f"cut -f 1 {hightemp}"), 'not match col1'
with open(col2_f, 'r') as f:
    assert f.read() + '\n' == system(f"cut -f 2 {hightemp}"), 'not match col2'

In [11]:
# not readlines
col1_f = 'data/02-unix/col1.txt'
col2_f = 'data/02-unix/col2.txt'
with open(hightemp) as f, open(col1_f, 'w') as col1_w, open(col2_f, 'w') as col2_w:
    for line in f:
        col1, col2, *rest = line.split('\t')
        col1_w.write(f'{col1}\n')
        col2_w.write(f'{col2}\n')

with open(col1_f, 'r') as f:
    assert f.read() == system(f"cut -f 1 {hightemp}"), 'not match col1'
with open(col2_f, 'r') as f:
    assert f.read() == system(f"cut -f 2 {hightemp}"), 'not match col2'

13. col1.txtとcol2.txtをマージ
12で作ったcol1.txtとcol2.txtを結合し，元のファイルの1列目と2列目をタブ区切りで並べたテキストファイルを作成せよ．確認にはpasteコマンドを用いよ．

In [19]:
col12 = 'data/02-unix/col12.txt'
with open(col1_f, 'r') as col1_r, open(col2_f, 'r') as col2_r, open(col12, 'w') as w:
    for col1, col2 in zip(col1_r, col2_r):
        w.write(f'{col1[:-1]}\t{col2[:-1]}\n')

assert system(f'cat {col12}') == system(f"paste  {col1_f} {col2_f}"), 'not match'

14. 先頭からN行を出力
自然数Nをコマンドライン引数などの手段で受け取り，入力のうち先頭のN行だけを表示せよ．確認にはheadコマンドを用いよ．

In [30]:
import argparse

# Nをコマンドライン引数として受け取る
def create_parser():
    parser = argparse.ArgumentParser(description='N')
    parser.add_argument('--N', type=int, default=10,
                        help='Number of line to show')
    return parser
parser = create_parser()
args = parser.parse_args(args=['--N', '10'])
N = args.N

with open(hightemp) as f:
    text = f.read()

n_lines = '\n'.join(text.split('\n')[:N])

assert n_lines + '\n' == system(f"head -n {N} {hightemp}"), 'not match'


15. 末尾のN行を出力
自然数Nをコマンドライン引数などの手段で受け取り，入力のうち末尾のN行だけを表示せよ．確認にはtailコマンドを用いよ．

In [31]:
# 以下、Nのコマンドライン引数としての受け取りを省略
N = 10

with open(hightemp) as f:
    text = f.read()[:-1]

n_lines = '\n'.join(text.split('\n')[-N:])

assert n_lines + '\n' == system(f"tail -n {N} {hightemp}"), 'not match'


16. ファイルをN分割する
自然数Nをコマンドライン引数などの手段で受け取り，入力のファイルを行単位でN分割せよ．同様の処理をsplitコマンドで実現せよ．


In [32]:
N = 10
with open(hightemp) as f:
    text = f.read()[:-1]
lines = text.split('\n')
line_count = len(lines)
if line_count % N == 0:
    lines_ave = line_count // N
else:
    lines_ave = line_count // N + 1
split_points = [lines_ave*i for i in range(N+1)]
split_points[N] = line_count

for i in range(N):
    with open(f'./data/02-unix/split_python/{i}.txt', 'w') as f:
        f.write('\n'.join(lines[split_points[i]: split_points[i+1]]))
command = f'''
n=10
# wcの出力の先頭に空白が６つある
count=`wc -l {hightemp} | cut -f 7 -d " "`

if [ $(($count % $n)) = 0 ]; then
    unit=$(($count / $n))
else
    unit=$(($count / $n + 1))
fi

# 分割
cd data/02-unix/split_unix
split -l $unit  ~/googledrive/PycharmProjects/nlp100/data/02-unix/hightemp.txt
'''
system(command)
# test は見比べることにする


''

17. １列目の文字列の異なり
1列目の文字列の種類（異なる文字列の集合）を求めよ．確認にはsort, uniqコマンドを用いよ．


In [33]:
with open(hightemp) as f:
    text = f.read()[:-1]
lines = text.split('\n')
col1 = [line.split('\t')[0] for line in lines]
eles_1 = set(col1)
command = f'cut -f 1 {hightemp} | sort | uniq'
assert eles_1 == set(system(command)[:-1].split('\n')), 'not match'


18. 各行を3コラム目の数値の降順にソート
各行を3コラム目の数値の逆順で整列せよ（注意: 各行の内容は変更せずに並び替えよ）．確認にはsortコマンドを用いよ（この問題はコマンドで実行した時の結果と合わなくてもよい）．


In [34]:
with open(hightemp) as f:
    text = f.read()[:-1]
cols = [line.split('\t')[0] for line in lines]
sorted_lines = sorted(lines, key=lambda line: float(line.split('\t')[2]), reverse=True)

command = f'sort {hightemp} --key=3 --numeric-sort --reverse'
print('\n'.join(sorted_lines))
print(system(command))
# assert '\n'.join(sorted_lines) + '\n' == system(command), 'not match'

高知県	江川崎	41	2013-08-12
埼玉県	熊谷	40.9	2007-08-16
岐阜県	多治見	40.9	2007-08-16
山形県	山形	40.8	1933-07-25
山梨県	甲府	40.7	2013-08-10
和歌山県	かつらぎ	40.6	1994-08-08
静岡県	天竜	40.6	1994-08-04
山梨県	勝沼	40.5	2013-08-10
埼玉県	越谷	40.4	2007-08-16
群馬県	館林	40.3	2007-08-16
群馬県	上里見	40.3	1998-07-04
愛知県	愛西	40.3	1994-08-05
千葉県	牛久	40.2	2004-07-20
静岡県	佐久間	40.2	2001-07-24
愛媛県	宇和島	40.2	1927-07-22
山形県	酒田	40.1	1978-08-03
岐阜県	美濃	40	2007-08-16
群馬県	前橋	40	2001-07-24
千葉県	茂原	39.9	2013-08-11
埼玉県	鳩山	39.9	1997-07-05
大阪府	豊中	39.9	1994-08-08
山梨県	大月	39.9	1990-07-19
山形県	鶴岡	39.9	1978-08-03
愛知県	名古屋	39.9	1942-08-02
高知県	江川崎	41	2013-08-12
岐阜県	多治見	40.9	2007-08-16
埼玉県	熊谷	40.9	2007-08-16
山形県	山形	40.8	1933-07-25
山梨県	甲府	40.7	2013-08-10
静岡県	天竜	40.6	1994-08-04
和歌山県	かつらぎ	40.6	1994-08-08
山梨県	勝沼	40.5	2013-08-10
埼玉県	越谷	40.4	2007-08-16
群馬県	館林	40.3	2007-08-16
群馬県	上里見	40.3	1998-07-04
愛知県	愛西	40.3	1994-08-05
静岡県	佐久間	40.2	2001-07-24
愛媛県	宇和島	40.2	1927-07-22
千葉県	牛久	40.2	2004-07-20
山形県	酒田	40.1	1978-08-03
群馬県	前橋	40	2001-07-24
岐阜県	美濃	40	2007-08-16
愛知県	名古屋	39.9	1942-08-02
山梨県	大

19. 各行の1コラム目の文字列の出現頻度を求め，出現頻度の高い順に並べる
各行の1列目の文字列の出現頻度を求め，その高い順に並べて表示せよ．確認にはcut, uniq, sortコマンドを用いよ．


In [34]:
with open(hightemp) as f:
    text = f.read()[:-1]
lines = text.split('\n')
eles_1 = [line.split('\t')[0] for line in lines]
ele1_dict = {ele: 0 for ele in set(eles_1)}
for ele in eles_1:
    ele1_dict[ele] += 1

sorted_eles = sorted(ele1_dict.items(), key=lambda item: int(item[1]), reverse=True)
sorted_eles = '\n'.join(f'{item[0]}\t{item[1]}' for item in sorted_eles)
print(sorted_eles)
command = f"cut -f 1 {hightemp} | sort | uniq -c | sort -n -r | cut -f 5 -d ' '"
print(system(command))
# assert sorted_eles + '\n' == system(command), 'not match'

山梨県	3
山形県	3
群馬県	3
埼玉県	3
岐阜県	2
愛知県	2
千葉県	2
静岡県	2
大阪府	1
高知県	1
愛媛県	1
和歌山県	1
群馬県
山梨県
山形県
埼玉県
静岡県
愛知県
岐阜県
千葉県
和歌山県
高知県
愛媛県
大阪府

